# How to the TextVectorization Layer in TensorFLow (TensorFLow'da Metin Vektörleştirme Katmanı Nasıl Oluşturulur)

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [2]:
# Instanting
text_vectorization = TextVectorization()

In [3]:
data = [
    "Bugün hava çok güzel",
    "Ali, Efe ve Ece çay içecek",
    "Selam söyle"
]

In [4]:
# Creating the vocabulary with the adapt method.
text_vectorization.adapt(data)

In [5]:
# Let's take a look at the vocabulary.
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 np.str_('çok'),
 np.str_('çay'),
 np.str_('ve'),
 np.str_('söyle'),
 np.str_('selam'),
 np.str_('içecek'),
 np.str_('hava'),
 np.str_('güzel'),
 np.str_('efe'),
 np.str_('ece'),
 np.str_('bugün'),
 np.str_('ali')]

In [6]:
# Data preprocessing with the layer
vectorized_text = text_vectorization(data)
vectorized_text

<tf.Tensor: shape=(3, 6), dtype=int64, numpy=
array([[12,  8,  2,  9,  0,  0],
       [13, 10,  4, 11,  3,  7],
       [ 6,  5,  0,  0,  0,  0]])>

3 cümle için 3 vektör oluşturuldu, ve kelimler indekslere göre sayısallaştırılıd. 
örneğin; 

Indeks saymaya 1'den başlanır. "[UNK]" sözlükte olmayan kelime içindir. 
- 12 -> 12. indeksteki kelime ("bugün")
- 8 -> 8. indeksteki kelime ("hava")

Bu kodda "0", cümle uzunluklarını eşitlemek için eklenen "padding" (doldurma) değerini ifade eder; yani gerçek bir kelimeye karşılık gelmez.

# Using the custom functions TextVectorization (Özel fonksiyonları kullanma TextVectorization)

In [6]:
import re
import string

In [7]:
def standardization_fn(string_tensor):
  lowercase=tf.strings.lower(string_tensor)
  return tf.strings.regex_replace(
      lowercase, f"[{re.escape(string.punctuation)}]", ""
  )

In [8]:
def split_fn(string_tensor):
  return tf.strings.split(string_tensor)

In [9]:
text_vectorization = TextVectorization(
    standardize=standardization_fn,
    split = split_fn
)

In [10]:
text_vectorization.adapt(data)

In [11]:
# Testing our layer with a text
text = "bugün ece çok güzel"
text_vectorization(text)

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([12, 11,  2,  9])>

# Using TextVectorization in a model (TextVectorization'ı bir modelde kullanma)

Bu katman normalde CPU'da çalışır. Bu katman, tek başına kullanılırsa, modelimiz GPU'yu kullansa bile bu fonksiyon CPU'da çalışacağı için eğitim yavaş olur. Bu katmanı GPU'da çalıştırmak için ya model içerisinde ya da data pipeline (veri işleme pipeline) içerisine yerleştirilmelidir.

In [13]:
import tensorflow as tf

print("GPU kullanılıyor mu?:", tf.config.list_physical_devices('GPU'))

GPU kullanılıyor mu?: []


Eğer çıktı boş bir liste ise ([]), GPU kullanılmıyor, yani CPU'dayız

In [14]:
# Creating a Dataset object
text_dataset = tf.data.Dataset.from_tensor_slices([
    "kedi", "aslan", "yunus"
])

In [ ]:
# Creating the TextVectorization layer for pre-processing 
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=5000, # token sayısını sınırlama 
    output_sequence_length=4  # padding yapmak için dizi uzunluğu
)

In [ ]:
# Creating the vocabulary
vectorize_layer.adapt(text_dataset.batch(64)) # .batch(64) büyük veriler için tercih edilir 

In [17]:
vectorize_layer.get_vocabulary()

['', '[UNK]', np.str_('yunus'), np.str_('kedi'), np.str_('aslan')]

In [18]:
# Building the model
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer
])

In [ ]:
import numpy as np

input_data = np.array(["kedi kartal aslan", "fok yunus"], dtype=object) 
# Burada, input_data değişkenine metinleri içeren bir NumPy dizisi oluşturuluyor ve dtype=object ile her elemanın Python nesnesi (string gibi) olarak saklanması sağlanıyor.

In [34]:
model.predict(input_data)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


array([[3, 1, 4, 0],
       [1, 2, 0, 0]])

"kartal", 1 olarak indekslenmiş, çünkü sözlükte yok
"kedi", 3 olarak indekslenmiş, çünkü sözlükte 3 
"fok", 1 olarak indekslenmiş, çünkü sözlükte yok
0 indeksleri padding için